In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import multiprocessing as mp

import LGTp as lgt

# Review of Extending machine learning classification capabilities with histogram reweighting  
  
ref : [Bachtis, Aarts, Lucini, Phys. Rev. E 102, 033303](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.102.033303)

This note is to review and reproduce phase detection using machine learning with histogram reweighting.  It will be done by following procedure.  
1. We define Ising model action and generate Ising model configurations using Wolff algorithm for traning and validation.  
2. Build CNN layers proposed in the ref and train using the Ising model configurations
3. Summarize about histogram reweighting and apply it to CNN output

In [2]:
def calcEnergy(config):
    """Calculate energy density
    """
    energy = 0
    
    for i in range(len(config)):
        for j in range(len(config)):
            s = config[i,j]
            nn = config[(i+1)%N,j] \
            + config[(i-1)%N,j] \
            + config[i,(j+1)%N] \
            + config[i,(j-1)%N]
            energy += -nn*s
    return energy*0.5/len(config)/len(config)

def calcMag(config):
    """Calculate magnetization
    """
    mag = np.average(config)
    return mag

def calcAbsMag(config):
    """Calculate absolute value of magnetization
    """
    mag = np.abs(np.average(config))
    return mag

In [3]:
def detect_ac_t(ac, tol = 5e-2, step = 3):
    
    t = len(ac)
    
    for i in range(len(ac) - step):
        
        diff = np.average(np.abs(ac[i:i+step]))
        
        if diff < tol:
            t = i
            return t
    
    return t

In [4]:
def autocorrelation(conf, O, t):
    
    N = len(conf) - t
    
#     cor_hist = np.empty(N)
    
    o1o2 = 0.
    o1 = 0.
    o2 = 0.
    
    o1o1 = 0.
    o2o2 = 0.
    
    for i in range(N):
        
        o1o2 += O(conf[i])*O(conf[i+t])/N
        
        o1 += O(conf[i])/N
        o2 += O(conf[i+t])/N
        
        o1o1 += O(conf[i])*O(conf[i])/N
        o2o2 += O(conf[i+t])*O(conf[i+t])/N
        
    o1_err = np.sqrt(o1o1 - o1*o1)
    o2_err = np.sqrt(o2o2 - o2*o2)
        
    cor_t = o1o2 - o1*o2
        
    return cor_t/o1_err/o2_err

## 1. Ising model action and 2d lattice object

In [5]:
# Equilibrating phase detection
def calc_teq(bare_arg, O, tol=5e-2, step=1000,seed=0):
    t_eq = step

    O_cold = []
    O_hot = []

    ising_cold = lgt.Lattice([N,N])
    ising_cold.init_fields('Ising2d','Cold',seed)

    ising_hot = lgt.Lattice([N,N])
    ising_hot.init_fields('Ising2d','Hot',seed)
    
    for i in range(step):
        lgt.metropolis(ising_cold,bare_arg)
        lgt.metropolis(ising_hot,bare_arg)

        diff = np.abs(O(ising_cold.field) - O(ising_hot.field))
        O_cold.append(O(ising_cold.field))
        O_hot.append(O(ising_hot.field))

        if diff < tol:
            t_eq = i
            break
    
    return t_eq

In [6]:
def calc_tac(bare_arg, t_eq, tol=5e-2, steps = 500,seed=0):
    
    ising_ac = lgt.Lattice([N,N])
    ising_ac.init_fields('Ising2d','Cold',seed)

    n_equi_ac = t_eq
    for i in range(n_equi_ac):
        lgt.metropolis(ising_ac, bare_arg)

    n_conf_ac = steps
    conf_ac = []
    M_ac = []
    for e in range(n_conf_ac):
        lgt.metropolis(ising_ac,bare_arg)
        conf_ac.append(ising_ac.field)
        Mag = calcMag(ising_ac.field)
        M_ac.append(Mag)
        
    ac_hist = np.empty(len(conf_ac))

    for i in range(len(conf_ac)):
        ac_hist[i] = autocorrelation(conf_ac,calcAbsMag,i)

    t_ac = detect_ac_t(ac_hist) + 1
    
    return t_ac

In [7]:
# started 0 increasing by 1
# beta1 = np.arange(0.32,0.41,0.01)
# beta2 = np.arange(0.47,0.56,0.01)
# beta_list = np.r_[beta1,beta2]
beta_list = np.arange(0.434,0.448,0.002)

nt = len(beta_list)

ensem = []
N = 128
n_conf = 1000

def simulate(b):
# for b in beta_list:
    seed = int(beta_list[b]*1000)
#     print("\n=================\n")
#     print("* Beginig beta : ",b)
#     print("-----------------\n")
    bare_parameters = {'beta':beta_list[b], 'J':1., 'h':0.00, 'mu':0.1}
    
#     print("* Pre-MC stage 1 : calculating t_eq")
    t_eq = calc_teq(bare_parameters,calcAbsMag,seed=seed)
#     print("equilibrium steps t_eq : ",t_eq)
#     print("-----------------\n")
    
#     print("* Pre-MC stage 2 : calculating t_ac")
    t_ac = calc_tac(bare_parameters, t_eq,seed=seed)
#     print("autocorrelation steps t_a : ",t_ac)
#     print("-----------------\n")
    
    ising = lgt.Lattice([N,N])
    ising.init_fields('Ising2d','Cold',seed)
    
#     print("* Pre-MC stage 3 : equilibrating phase")
    for i in range(t_eq):
        lgt.metropolis(ising,bare_parameters)
#     print("------done-------\n")

    conf = []    
    
#     print("* MC stage")
#     print("Generating ",n_conf," configurations.")
    
    for e in range(n_conf*t_ac):

        lgt.metropolis(ising,bare_parameters)
        
        if not e%t_ac:
            
            conf.append(ising.field)
            
#     print("------done-------\n")
    conf_name = './data/Ising128/Ising_b%0.3fN%d' %(bare_parameters['beta'],N)
    np.save(conf_name+'.npy', conf)



In [ ]:
start = time.time()

p = mp.Pool(4)
res = p.map(simulate, range(nt))
p.close()
p.join()

end = time.time() - start
print("time span : ",end)

/tmp/ipykernel_9233/3067718072.py:29: RuntimeWarning: invalid value encountered in double_scalars
  return cor_t/o1_err/o2_err
/tmp/ipykernel_9233/3067718072.py:29: RuntimeWarning: invalid value encountered in double_scalars
  return cor_t/o1_err/o2_err
/tmp/ipykernel_9233/3067718072.py:29: RuntimeWarning: invalid value encountered in double_scalars
  return cor_t/o1_err/o2_err
/tmp/ipykernel_9233/3067718072.py:29: RuntimeWarning: invalid value encountered in double_scalars
  return cor_t/o1_err/o2_err
